<a href="https://colab.research.google.com/github/menwcode/Data-Science/blob/main/pyspark_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [ ]:
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

In [ ]:
#list all the files
!ls -l

total 214788
drwxr-xr-x  1 root root      4096 Nov  6 17:30 sample_data
drwxr-xr-x 13 1000 1000      4096 Aug 28 08:10 spark-3.0.1-bin-hadoop2.7
-rw-r--r--  1 root root 219929956 Aug 28 09:25 spark-3.0.1-bin-hadoop2.7.tgz


In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

In [ ]:
!wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD

In [ ]:
!ls -l

In [ ]:
#move the rows.csv?accessType=DOWNLOAD
!mv rows.csv\?accessType\=DOWNLOAD reported-crimes.csv


In [ ]:
!ls -l

In [ ]:
#Laod data in pyspark
from pyspark.sql.functions import to_timestamp, col, lit
df = spark.read.csv('reported-crimes.csv', header = True).withColumn('Date', to_timestamp(col('Date'), 'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') <= lit('2018-11-11'))
df.show(5)

In [ ]:
##List all the columns
df.columns

In [ ]:
#Length of the all the fields
len(df.columns)

In [ ]:
#Count method to get the all the records
df.count()

In [ ]:
#A duplicate of Pandas shape
print((df.count()), (len(df.columns)))

In [ ]:
#List the Schema
df.printSchema()

In [ ]:
#show only 2 columns values
df.select('Case Number', 'Arrest').show(5)

In [ ]:
#Statistical measure of each columns of the dataframe
df.describe().show()

In [ ]:
#Adding a new column
#withcolumn function of Spark, lets you add a new column  sexual_crime by checking "Primary Type"
#df.withColumn("sexual_crime")


In [ ]:
from pyspark.sql.types import StringType, DoubleType

Filtering data


In [ ]:
df.filter(df["Primary Type"] == "CRIM SEXUAL ASSAULT").show()

In [ ]:
#Show all the crimes related to Sexual assulat and Predatory
df.filter((df["Primary Type"] == "CRIM SEXUAL ASSAULT") & (df["Description"] == "PREDATORY")).show()


Show Distinct Values

In [ ]:
#Show distinct values of with Truncate set to False
df.select("Primary Type").distinct().show(truncate = False)

In [ ]:
#Grouping Data
df.groupBy('Primary Type').count().show(50, False)

Put order in above output

In [ ]:
df.groupBy("Primary Type").count().orderBy('count', ascending=False).show(50, False)

In [ ]:
df.groupBy("Block").count().orderBy('count', ascending=False).show(50, False)